In [8]:
import numpy as np
import scipy
import scipy.linalg

![/home/zhuyekun/data/Scientific-Computing/hw1/pro3.JPG](./1.JPG)

![/home/zhuyekun/data/Scientific-Computing/hw1/pro3.JPG](./2.JPG)

![/home/zhuyekun/data/Scientific-Computing/hw1/pro3.JPG](./3.JPG)

# T1 
Computer problem (in C or C++): Using Gaussian elimination to achieve the LU decomposition with and without a column pivoting; Using the two LU decomposition algorithm to solve linear systems in which the coefficient matrix is (1) general nonsingular matrix; (2) positive definite matrix; (3) diagonally dominant matrix. Compare the numerical accuracy for the two algorithms. The size of the matrices should be greater than 1000.

In [9]:
def Q_i(Q_min, i, j, k):
    if i < k or j < k:
        return float(i == j)
    else:
        return Q_min[i-k][j-k]

def QR_householder(A):
    n = A.shape[0]

    # Set R equal to A, and create Q as a zero matrix of the same size
    R = A.copy()
    Q = np.zeros((n,n))

    # The Householder procedure
    for k in range(n-1):
        I = np.eye(n)
        x = R[k:,k]
        e = I[k:,k]
        alpha = np.sign(x[0]) * np.linalg.norm(x)

        # Using anonymous functions, we create u and v
        u = np.array(list(map(lambda p,q: p + alpha * q, x, e)))
        # print(u)
        norm_u = np.linalg.norm(u)
        v = np.array(list(map(lambda p: p/norm_u, u)))

        # Create the Q minor matrix
        Q_min = np.array([ [float(i==j) - 2.0 * v[i] * v[j] for i in range(n-k)] for j in range(n-k) ])

        # "Pad out" the Q minor matrix with elements from the identity
        Q_t = np.array([[ Q_i(Q_min,i,j,k) for i in range(n)] for j in range(n)])

        if k == 0:
            Q = Q_t
            R = Q_t@A
        else:
            Q = Q_t@Q
            R = Q_t@R

    # Since Q is defined as the product of transposes of Q_t,
    # we need to take the transpose upon returning it
    return Q.T, R

In [10]:
def QR_Household_column_pivot(A_x):
    A = A_x
    A_star = A_x
    Q = np.eye(A.shape[0])
    R = np.zeros((A.shape[0],A.shape[1]))
    P = np.eye(A.shape[0])
    for k in range(A.shape[0]):
        argmax_k = np.argmax(np.abs(A[k:,k]))
        P_k = np.eye(A.shape[0])
        P_k[[k,argmax_k+k],:] = P_k[[argmax_k+k,k],:] 
        A = P_k@A
        a_k = - np.sign(A[k][k])*np.sqrt(np.sum(A[k:,k]**2))
        v_k = np.reshape(np.concatenate((np.zeros((k)),A[k:,k])),(np.shape(A)[0],1)) - a_k * np.reshape(np.concatenate((np.zeros((k)),np.ones((1)),np.zeros((np.shape(A)[0]-1-k)))),(np.shape(A)[0],1))
        beta_k = np.transpose(v_k)@v_k
        if beta_k == 0:
            continue
        for j in range(k,A.shape[1]):
            gamma_j = np.transpose(v_k)@A[:,j]
            A[:,j] = A[:,j] - np.reshape((2*gamma_j/beta_k)*v_k,np.shape(A)[0])
        H = np.eye(A.shape[0]) - 2 / beta_k * v_k@np.transpose(v_k)
        A_star = P_k@A_star
        A_star = H@A_star
        Q = Q@np.transpose(P_k)
        Q = Q@np.transpose(H)

    return Q , A

# 验证

In [11]:
A = np.array([[12, -51, 4], [6, 167, -68], [-4, 24, -41]])
Q, R = QR_householder(A)

In [14]:
print("Q,R分别为：",Q,R)

Q,R分别为： [[-0.85714286  0.39428571  0.33142857]
 [-0.42857143 -0.90285714 -0.03428571]
 [ 0.28571429 -0.17142857  0.94285714]] [[-1.40000000e+01 -2.10000000e+01  1.40000000e+01]
 [-5.57673565e-16 -1.75000000e+02  7.00000000e+01]
 [-5.08994556e-16 -7.64989650e-16 -3.50000000e+01]]


In [15]:
q,r = scipy.linalg.qr(A)
print("Q,R与真实的Q,R对比为：",Q-q,R-r)

Q,R与真实的Q,R对比为： [[ 2.22044605e-16 -1.66533454e-16 -5.55111512e-17]
 [ 5.55111512e-17  2.22044605e-16 -4.85722573e-17]
 [-5.55111512e-17 -2.77555756e-17  0.00000000e+00]] [[ 1.77635684e-15  0.00000000e+00 -3.55271368e-15]
 [-5.57673565e-16  5.68434189e-14 -2.84217094e-14]
 [-5.08994556e-16 -7.64989650e-16  0.00000000e+00]]
